# 联邦学习

## 导入数据

In [1]:
import numpy as np
from collections import Counter
import random
import sys
np.random.seed(12345)

# dataset from http://www2.aueb.gr/users/ion/data/enron-spam/

import codecs
with codecs.open('spam.txt', "r",encoding='utf-8', errors='ignore') as fdata:
    raw = fdata.readlines()

vocab = set()
    
spam = list()
for row in raw:
    spam.append(set(row[:-2].split(" ")))
    for word in spam[-1]:
        vocab.add(word)
    
import codecs
with codecs.open('ham.txt', "r",encoding='utf-8', errors='ignore') as fdata:
    raw = fdata.readlines()

ham = list()
for row in raw:
    ham.append(set(row[:-2].split(" ")))
    for word in ham[-1]:
        vocab.add(word)
        
vocab.add("<unk>")

vocab = list(vocab)
w2i = {}
for i,w in enumerate(vocab):
    w2i[w] = i
    
def to_indices(input, l=500):
    indices = list()
    for line in input:
        if(len(line) < l):
            line = list(line) + ["<unk>"] * (l - len(line))
            idxs = list()
            for word in line:
                idxs.append(w2i[word])
            indices.append(idxs)
    return indices


In [2]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None

        if(id is None):
            self.id = np.random.randint(0,1000000000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    return
                    print(self.id)
                    print(self.creation_op)
                    print(len(self.creators))
                    for c in self.creators:
                        print(c.creation_op)
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if(self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if(self.autograd):
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
        
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

class Layer(object):
    
    def __init__(self):
        self.parameters = list()
        
    def get_parameters(self):
        return self.parameters

    
class SGD(object):
    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
        
    def step(self, zero=True):
        
        for p in self.parameters:
            
            p.data -= p.grad.data * self.alpha
            
            if(zero):
                p.grad.data *= 0


class Linear(Layer):

    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        
        self.use_bias = bias
        
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        if(self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        
        if(self.use_bias):        
            self.parameters.append(self.bias)

    def forward(self, input):
        if(self.use_bias):
            return input.mm(self.weight)+self.bias.expand(0,len(input.data))
        return input.mm(self.weight)


class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
    
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        # this random initialiation style is just a convention from word2vec
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5) / dim, autograd=True)
        
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)


class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()


class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()
    

class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        return input.cross_entropy(target)

class MSELoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        dif = input - target
        return (dif * dif).sum(0)
    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
    
class LSTMCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)        
        self.xc = Linear(n_inputs, n_hidden)        
        
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)        
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()

        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()        
        self.parameters += self.ho.get_parameters()        
        self.parameters += self.hc.get_parameters()                
        
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        
        prev_hidden = hidden[0]        
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()        
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()        
        c = (f * prev_cell) + (i * g)

        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        init_hidden = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_cell = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_hidden.data[:,0] += 1
        init_cell.data[:,0] += 1
        return (init_hidden, init_cell)

## 学习检测垃圾邮件

In [3]:
spam_idx = to_indices(spam)
ham_idx = to_indices(ham)

train_spam_idx = spam_idx[0:-1000]
train_ham_idx = ham_idx[0:-1000]

test_spam_idx = spam_idx[-1000:]
test_ham_idx = ham_idx[-1000:]

train_data = list()
train_target = list()

test_data = list()
test_target = list()

for i in range(max(len(train_spam_idx),len(train_ham_idx))):
    train_data.append(train_spam_idx[i%len(train_spam_idx)])
    train_target.append([1])
    
    train_data.append(train_ham_idx[i%len(train_ham_idx)])
    train_target.append([0])
    
for i in range(max(len(test_spam_idx),len(test_ham_idx))):
    test_data.append(test_spam_idx[i%len(test_spam_idx)])
    test_target.append([1])
    
    test_data.append(test_ham_idx[i%len(test_ham_idx)])
    test_target.append([0])

    
def train(model, input_data, target_data, batch_size=100, iterations=5):
    n_batches = int(len(input_data) / batch_size)
    for iteration in range(iterations):
        iter_loss = 0
        for b_i in range(n_batches):
            (batch_start, batch_end) = (b_i * batch_size, (b_i+1) * batch_size)
            model.weight.data[w2i['<unk>']] *= 0
            input = Tensor(input_data[batch_start: batch_end], autograd=True)
            target = Tensor(target_data[batch_start: batch_end], autograd=True)
            
            pred = model.forward(input).sum(1).sigmoid()
            loss = criterion.forward(pred, target)
            loss.backward()
            optim.step()
            
            iter_loss += loss.data[0]
            sys.stdout.write('\r\tLoss:' + str(iter_loss / batch_end))
        print()
    return model

def test(model, test_input, test_output):
    model.weight.data[w2i['<unk>']] *= 0
    
    input = Tensor(test_input, autograd=True)
    target = Tensor(test_output, autograd=True)
    
    pred = model.forward(input).sum(1).sigmoid()
    return ((pred.data > 0.5) == target.data).mean()

In [4]:
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.1)

for i in range(3):
    model = train(model, train_data, train_target, iterations=1)
    print('% Correct on Test Set:' + str(test(model, test_data, test_target) * 100))

	Loss:0.010722979357422576
% Correct on Test Set:99.75
	Loss:0.0021028410557890286
% Correct on Test Set:100.0
	Loss:0.0011595075974399492
% Correct on Test Set:100.0


## 联邦化 

In [5]:
bob = (train_data[0:1000], train_target[0:1000])
alice = (train_data[1000:2000], train_target[1000:2000])
sue = (train_data[2000:], train_target[2000:])

In [6]:
import copy
for i in range(3):
    print('Starting Training Round...')
    print('\tStep1: send the model to bob')
    bob_model = train(copy.deepcopy(model), bob[0], bob[1], iterations=1)
    
    print('\tStep2: send the model to alice')
    alice_model = train(copy.deepcopy(model), alice[0], alice[1], iterations=1)
    
    print('\tStep3: send the model to sue')
    sue_model = train(copy.deepcopy(model), sue[0], sue[1], iterations=1)
    
    print('\n\tAverage Everyone\'s New Models ')
    model.weight.data = (bob_model.weight.data + alice_model.weight.data + sue_model.weight.data) / 3
    print("\t% Correct on Test Set: " + \
          str(test(model, test_data, test_target)*100))
    print('\n Repeat!\n')
    

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176152
	Step2: send the model to alice
	Loss:0.0009465600605861596
	Step3: send the model to sue
	Loss:0.0009149338888900243

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176152
	Step2: send the model to alice
	Loss:0.0009465600605861595
	Step3: send the model to sue
	Loss:0.0009149338888900241

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!

Starting Training Round...
	Step1: send the model to bob
	Loss:0.0011451959675176152
	Step2: send the model to alice
	Loss:0.0009465600605861595
	Step3: send the model to sue
	Loss:0.0009149338888900241

	Average Everyone's New Models 
	% Correct on Test Set: 100.0

 Repeat!



### 深入联邦学习
* 隐私问题——从权重更新得知数据的内容

In [8]:
import copy

bobs_email = ["my", "computer", "password", "is", "pizza"]

bob_input = np.array([[w2i[x] for x in bobs_email]])
bob_target = np.array([[0]])

# model = Embedding(vocab_size=len(vocab), dim=1)
# model.weight.data *= 0

bobs_model = train(copy.deepcopy(model), bob_input, bob_target, iterations=1, batch_size=1)

	Loss:0.08395573209431818


In [9]:
for i,v in enumerate(bob_model.weight.data - model.weight.data):
    if v is not 0:
        print(vocab[i])


complementary
olefin
background
sir
acsdallas
fermium
selcydc
buchwald
tosco
beaches
piayers
abcdzhongguo
igb
gpa
verantwortlich
requistions
giant
sagging
othe
npukl
immigration
newsstands
therewhen
stivers
barrister
roemco
css
rslku
delights
????????
qxtnqdm
mr
eiectricity
cqqc
4473
ykl
moopid
txtnppm
downs
epithelial
365
disclaims
smyth
djjdnj
susie
257144
comorsend
antisemitic
cigars
intentional
biz
icgdkxu
fof
susb
absolutionelision
schoppe
coerce
off
27099
ciceronian
juggins
sarah
2578
gaps
govenment
serving
round
ries
quo
bollom
luuuuuuube
boyce
akers
spray
directories
fuifiil
courier
96029892
honorably
kmay
ease
retubing
forty
yuga
collect
kmojp
ofa
montagne
disclosed
malheiro
ksssu
gestohlenen
stevedore
rjqhcz
ctc
bladder
socratic
kodak
drift
marceau
comc
iyzxs
utiiize
pnxry
immortal
divisional
cook
giaex
carols
gegenschein
wearied
hastings
meridethtrafford
somal
714540
guygmyhv
repaired
encyclopedia
7852
wisp
examining
beyond
dogfish
fatiguecapitulate
237415
assigning
rpiebkw

persist
summertime
polypropylene
iodine
personally
qrjluyzdcuzfu
durch
executive
sancho
cinnabar
idiocy
fraser
lebanese
xnwcwsqxf
yoda
reliability
jesters
appreciated
uwfzct
sao
cjvucn
ben
fgk
secondly
ations
schroeder
syndyasmian
restart
schottky
krysta
monetizes
saloonkeep
attendee
neches
141883
carbamate
procees
dunk
zpbs
inattentive
ppv
nnkaro
volts
cps
space
florist
xwstuha
rjttzwso
ccgm
bbwciyuo
rainbowboatl
aric
frogsrugby
lehrer
dayof
garrett
social
pirates
emoej
free
millionth
commitment
polemic
aspirate
crow
avcp
valcourt
rumbling
corporate
cuokn
ewklfc
predictable
eader
calculate
eclat
jebgxoyxd
ssr
chicks
rwjlxrqv
prescribed
cowgirl
surfgrl
janie
foreclosures
medial
proliferate
jyymemrx
focusing
creditcards
hideous
ayad
xga
dutchman
enought
396904
beachcomb
fchoje
huang
cinergy
scans
poorer
2293
shipments
jocxitsemonioj
tzq
remain
mist
yawl
neurosis
ballet
suggests
mhzram
5160
stuckey
ionamin
mudix
hvfft
vlpmveb
idftkyjv
cara
demon
8862
convoyed
couch
97929
buick
stones
665

legislature
vlotspeich
phys
kalmick
diphthong
markhunt
44000
playoff
slide
clarissa
nne
shearing
zamit
3
arum
insofar
dizzytroposphere
knit
tally
mkktcxs
enthusiast
plachy
qua
lametrice
3165
hock
eqgas
goldman
1718
adela
asilomar
acp
settimeout
clothing
understood
angeies
quailty
8743
rut
ritchie
lana
30569
befell
tbshfn
withers
upgrading
208083
steps
buffet
109
salisbo
tish
statue
mkktcx
overflowed
uftkcuj
320
picnicked
602
specifying
77018
30000
ache
cipla
matters
deveioper
resigns
hearken
laval
wrap
epsom
homepage
orc
beplaster
dena
paxll
folliculin
appproved
nntixyhx
privacy
publisher
sta
cehczyox
today
remote
zzo
soupfin
95394
goldston
tract
kuhn
reviewing
heathenish
rht
identification
138086
qks
pentium
lore
fowls
greta
hecate
employers
montblanc
abbasi
discouraged
adjust
luzon
lucy
transco
hhmmj
sustantiate
coat
sforzando
qui
bootable
sued
rorewemove
expediasummer
winery
14
wolfgang
enfaa
tonsil
985801
60400
various
ukyk
hsv
wor
rbubav
milwaukee
mckeel
eachcollateral
naturalgain

knott
positive
few
liverpoo
144932
periferal
kratzer
recharge
contravene
pentagon
nox
kgav
diprotodont
udve
johann
down
charlton
handle
patel
jabverbiage
burrow
ts
betwixtl
ccheaper
zu
437
aron
centana
beadle
norfolk
episode
refined
crime
mcbeath
dubeau
milligan
oujkr
452582
327
ally
qbbcpryhrv
lymph
entex
asbill
duuai
let
pexik
picnic
torsion
subjectivity
bridgette
cheating
asymptotic
exig?
418436
apathy
comheip
839
herein
specificaliy
opportunites
bandstand
yhvqbvdboevkcd
aristocrat
daylight
buchanan
directly
booth
ovsr
deducted
soth
labor
centertrtdfont
vary
birk
tsavager
whose
000000000002858
qhf
abeckley
dreams
2174
vyou
oder
comply
2495
whatley
insomnia
mony
sid
nevertheless
spellbound
bayer
permitted
ceeuk
pacific
falerii
gates
systemworks
museums
chewa
accession
turban
pringle
exportable
stratosphere
porpoise
inelegant
delaying
cautioned
cdphfl
enp
proviso
sincerly
opening
presently
dewville
mcauliff
body
audiovisual
harvest
medium
depot
cialis
featuring
fwpgncv
competitors
cla

porterfields
sanicle
lydecker
pauli
codicil
azwybxnaw
analogy
cynet
vhf
retailers
cwkqlhix
mitchellgs
reisz
prizes
triphenylphosphine
athietic
inadvertently
christianson
relate
rasa
ciaiis
1921
darron
drawert
nzbsudpbpsjibwvfxskr
vmhx
yebdk
dpie
assist
233763
adjustements
<
oisv
newsvhxcwev
566
otherworld
cheapest
fxd
ordinary
article
neighbouring
daley
tuter
frisky
solemnity
phillipines
3369
apologies
evhchgf
piccolo
brbryour
teresa
palazzo
66277
chairmen
zhonda
expiode
643875
tehran
spark
kykubxxs
exxonkaty
yqzotn
gcgkmjcmeb
moreira
stokes
announcements
bryant
prerequisite
abercrombie
winner
dispersive
pioneers
mladen
4780
nwpuppy
u
salk
addend
waived
leagues
laquelle
bdfgads
issuance
telegraph
leadsman
nader
healer
wjw
tissue
nqozhica
individuate
ineffable
kopper
doubtful
carbon
37339
conqueror
handicraftsnbsp
repatriated
sha
1429
pursuer
275
378232683
&
ebjixj
positron
dream
eka
heterogamous
valuation
vlaggra
injuries
4664
emotions
980392
mastery
polyglot
eso
fertilizers
mobster
sh

...

err
msjx
ejya
periclean
aboardca
63000
haa
ccokinos
midstream
15600
stifle
incriminating
fibrosis
tuesdays
clpsjriw
gefesselt
congratulatory
uchinoyul
50791
skull
infrequent
piayer
iralsqasua
trim
apothegm
uzztyizoo
hex
0713
rti
aliquot
equip
cotty
markket
routers
iodide
surest
focal
coot
czar
geminate
emery
budgeted
stoxo
molina
veloz
986359
niccolo
alehc
021
johr
3643
misled
personality
pl
carboxybookend
284599
108058
urged
laurie
manner
adipren
vella
conduit
airport
36000
beakers
gigabitsquander
parameters
affix
incure
lump
jjqb
fraserpress
fanfare
vasectomy
dfxlqwdjz
credu
yoikgubbo
fungus
penetrations
sakula
9734
uvres
unfaithful
wu
typing
calculating
gqzzl
60217
looney
rick
pdnhjhlch
deferrals
held
myeasysavings
attentive
dow
oregon
lcr
bowfin
axiomatic
nlne
phentiermine
plt
jejohnson
spatula
lipi
horsewomen
madsen
their
815633
wounds
csicontrols
concerto
chalet
catalogue
ligne
138546
dramatic
porno
optimal
depositional
orienting
25000200
fwhpex
scan
36703
reieased
aaldano
provid

raspberry
lanxkxlvr
deeply
versace
wept
nuke
assignee
allspice
96029302
oz
eojs
moval
phrmcy
suspeneded
laurabarr
billow
912
nonsensic
adwords
cavil
tec
peusa
imperfect
alert
sdm
jeannie
ttmt
1411
telescope
138095
dukeadmiration
accedez
piraeus
octagonal
mastered
restaurant
seventy
xhtyca
frdllad
apostrophe
dramatically
hinder
sasha
gehzhxg
gnu
706
adults
allowing
lozano
comclick
bjfoysmvunbutuqk
transpose
agrarian
nicely
perscription
pineapple
coasters
commerciaiize
hoagburg
cape
dish
halff
refection
burg
valarie
evangelina
ptcontact
9825
vaiue
stagy
notion
ommerce
grimaldi
cbulganin
351
nous
enscoinc
provider
adult
75074
demo
0357
minnie
12782
51629
oitteodj
mauricio
gekommen
andiron
davenport
pbmelz
xfo
greenfield
zwu
ajntecdy
talk
lately
megonna
goliad
viia
capable
germannbsp
deprive
hobbys
bottom
gomes
vail
eukaryote
soiutions
hornblende
edith
maiden
proline
bursts
manchester
prdoucts
repair
spwtee
slte
disneyesque
registering
vx
lhacker
sasser
paradis
zlyprl
stenophones
proofs
em

statler
pearson
rices
jewkwevme
sakamkzzwtlsan
navarro
favour
ppjp
wzvybmze
hidden
layouts
mikie
styles
grain
douglass
faq
concessions
mp
1869
mve
regulate
counterargument
mhvcknl
reclasses
707274
vertical
8750
tag
delusive
prance
foxhall
032
5203
terminals
resign
752578
koskin
floyd
hideousness
bolometer
wide
whippanyparley
anguilliform
eyeball
4938
pjzbogw
end
ichor
hzohteeekqxcifbljt
clear
ugho
mellake
wx
bursty
gorilla
putted
lucking
gests
0042
864
bon
macy
deferent
halogen
filiing
gigccb
17738
wtyp
balmung
wmv
aec
andrew
awash
sneak
sib
transaktionen
badum
box
harrell
seasonal
bel
hype
scarlatti
yer
flagstaff
abused
ppt
guidelines
testing
quasicontinuous
triac
copeland
lamictal
ust
spencer
lm
burgomasters
accupuncture
cretin
dcp
corrine
sally
calibrate
wendell
zofran
anto
hydrocodoone
goff
hdrinc
inactive
tigris
voir
statement
eyed
cliente
icense
located
soaring
5163
edication
weak
chill
roundtrip
blurt
348951
hfoubzu
l?vy
transcended
61
afterbirth
arialour
porcine
woodrow
strings

989619
calcine
edgerton
arteriolosclerosis
welavo
alysia
jar
computers
shamefaced
chairlady
entertainers
proximate
6219
garage
captaino
awful
receiving
apyalis
aziz
florida
conn
gatekeep
413538
asparagine
prostitution
fad
afoot
vclw
bxfiurmpj
donindfw
holdings
reitmeyer
ulltram
eloise
acid
duvmalo
jibe
superorganize
shattering
frye
sasabihin
aging
demurrer
yael
blessing
advisors
oqaco
onlinegenericshop
successes
zero
imagine
iigp
qa
mead
qrbeatft
actively
doped
elliott
accident
boating
oxymomloree
gesture
tacitus
alis
woodworking
attacked
nudged
lowrate
oboist
powder
probusiness
dry
obtenir
lives
jarred
xnyrbdtwy
ovrnmhs
garnte
rbl
shell
facility
iceberg
debbiew
badland
numinous
qrtqqk
dkessler
delgado
alicia
wrdolwideyour
wiebe
share
coulter
priority
hidalgo
humping
genescoexaggerate
eclogue
arthritis
andesite
flaw
exceeding
champians
ox
optoisolate
sn
via
columbiagas
york
wrr
lidelivered
pie
responsiblefor
tumble
bloat
37913
tea
inherited
modules
pawnee
dietician
extensiblearesol
enr

butterfield
prague
120266
wrvydzdunplhogtjyoeqsq
casings
opposes
dialogue
squeegee
remv
scheming
charlielaeh
solitary
criticize
9002
morganj
load
oem
patio
cbzhn
amberhi
stiletto
callers
lastiendas
dreyfus
cipco
field
bespectacled
vings
828675346
viukj
quantifiable
4665
deaconess
goexpres
seekers
desecrate
am?rique
6105
hell
sneaked
zfrvfe
125827
loney
suburb
dynamism
deveiopment
sagebrush
dies
tpbvzbs
rmxjx
saxsuperb
success
overdelivery
diverting
kwmmdr
fpnurps
enjoy
ngpa
rounded
amazon
78581
101
bannantine
plastics
bastard
3949
clarified
rodeo
584
horrify
catalyst
dissension
positively
gaulle
acc?s
685
amend
savimbi
umsonst
39746
wordperfect
philips
pharmacopoeia
netherland
kacnrcp
animal
marten
naiveties
nbjrsm
powerline
bfxuqkgpv
zqt
vincent
osition
connotative
trayerrand
floodgate
nowlan
tritton
sewerage
6000
seminars
transmissions
atuliii
kneel
blvd
invi
millenium
melods
sodimm
lundell
jerald
fogy
jnelson
based
qfukrwbo
volpe
pam
xtghlwbcl
enrages
deraad
giantess
anastomosis
hon

## 安全聚合，同态加密

In [10]:
import phe

public_key, private_key = phe.generate_paillier_keypair(n_length=1024)

x = public_key.encrypt(5)

y = public_key.encrypt(3)

z = x + y

z_ = private_key.decrypt(z)
print("The Answer: " + str(z_))

The Answer: 8


In [11]:
public_key, private_key = phe.generate_paillier_keypair(n_length=128)

def train_and_encrypt(model, input, target, pubkey):
    new_model = train(copy.deepcopy(model), input, target, iterations=1)

    encrypted_weights = list()
    for val in new_model.weight.data[:,0]:
        encrypted_weights.append(public_key.encrypt(val))
    ew = np.array(encrypted_weights).reshape(new_model.weight.data.shape)
    
    return ew

In [12]:
for i in range(3):
    print("\nStarting Training Round...")
    print("\tStep 1: send the model to Bob")
    bob_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            bob[0], bob[1], public_key)

    print("\n\tStep 2: send the model to Alice")
    alice_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                              alice[0], alice[1], public_key)

    print("\n\tStep 3: Send the model to Sue")
    sue_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            sue[0], sue[1], public_key)

    print("\n\tStep 4: Bob, Alice, and Sue send their")
    print("\tencrypted models to each other.")
    aggregated_model = bob_encrypted_model + \
                       alice_encrypted_model + \
                       sue_encrypted_model

    print("\n\tStep 5: only the aggregated model")
    print("\tis sent back to the model owner who")
    print("\t can decrypt it.")
    raw_values = list()
    for val in sue_encrypted_model.flatten():
        raw_values.append(private_key.decrypt(val))
    model.weight.data = np.array(raw_values).reshape(model.weight.data.shape)/3

    print("\t% Correct on Test Set: " + \
              str(test(model, test_data, test_target)*100))


Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.0011451959675176152

	Step 2: send the model to Alice
	Loss:0.0009465600605861595

	Step 3: Send the model to Sue
	Loss:0.0009149338888900241

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 100.0

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.018781076276129308

	Step 2: send the model to Alice
	Loss:0.021959021980998947

	Step 3: Send the model to Sue
	Loss:0.020154686637448006

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 100.0

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.08585344314653713

	Step 2: send the model to Alice
	Loss:0.09230128994744338

	Step 3: Send the model to Sue
	Loss:0.0931310164